In [ ]:
import os 
import tensorflow as tf 

# Expects .tfrecords files as produced by the script in datasets in a google storage bucket
# Standard openwebtext 
# nhận đầu vào gồm params là một từ điển chứa các thông số cấu hình cho hàm như 
# data_path, batch_size , n_ctx . Tham số eval là một giá trị bool mặc định là false 
# xác định xem hàm này có được sử dụng cho mục đích đánh giá hay không . 
# tham số Batch cũng là một giá trị boolean mặc định True xác định xem hàm này có trả về một 
# đối tượng batch hay không
def openwebtext(params ,eval=False, batch=True):
    # nếu eval là False hàm này sẽ tạo một danh sách có tên là number, chứa các số 0->92
    # trừ 32 (vì 32 là trống) và một số khác. 
    if not eval:
        # các só trong danh mục numbers có chức năng là chỉ định các tệp tfrecord mà hàm openwebtext sẽ đọc 
        # các tệp tfrecord này được đặt tên theo dạng openwebtext-newspaper_{i}.tfrecord trong đó 
        # {i} là một số từ 0 -> 92  
        numbers = [0, 3, 4, 5, 6, 7, 8, 9, 10, 13, 14, 15, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, # 32, (32 is empty)
                33, 35, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 59, 63, 64,
                65, 66, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 84, 85, 86, 87, 88, 89, 90, 91, 92]
    # nếu Eval = True hàm này sẽ tạo một danh sách khác tên là numbers, chứa các số 
    # từ ->83 trừ một số khác .
    else:
        numbers = [1, 2, 11, 12, 16, 34, 36, 58, 60, 61, 62, 67, 83]

    # sau đó tạo một danh sách có tên là files bằng cách nối đường dẫn của các tệp tfrecords trong 
    # thư mục data_path của từ điển params với các số trong danh mục numbers . Các tệp tfrecord này chứa các văn bản 
    # từ nguồn OpenWebtext 
    files = [os.path.join(params['data_path'], "openwebtext-newspaper_{}.tfrecord" .format(str(i)))  for i in numbers]


    # cuối cùng hàm này sẽ gọi một hàm khác có tên là bpe_text với các tham số là batch_size 
    # files, amount, iterations, stitch và batch . Hàm này se sẽ trả về một danh sách văn bản được
    # mã hóa bằng phương pháp byte pair encoding 
    return bpe_text(params["batch_size"], files, amount=params["n_ctx"], iterations=params["iterations"], stitch=42, batch=batch)


# thiết lập một phương thức tương tự 
# nhưng chỉ lọc những văn bản có dài hơn 512 tokens 

def openwebtext_long(params ,eval=False, batch=True):
    # nếu eval là False hàm này sẽ tạo một danh sách có tên là number, chứa các số 0->92
    # trừ 32 (vì 32 là trống) và một số khác. 
    if not eval:
        # các só trong danh mục numbers có chức năng là chỉ định các tệp tfrecord mà hàm openwebtext sẽ đọc 
        # các tệp tfrecord này được đặt tên theo dạng openwebtext-newspaper_{i}.tfrecord trong đó 
        # {i} là một số từ 0 -> 92  
        numbers = [0, 3, 4, 5, 6, 7, 8, 9, 10, 13, 14, 15, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, # 32, (32 is empty)
                33, 35, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 59, 63, 64,
                65, 66, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 84, 85, 86, 87, 88, 89, 90, 91, 92]
    # nếu Eval = True hàm này sẽ tạo một danh sách khác tên là numbers, chứa các số 
    # từ ->83 trừ một số khác .
    else:
        numbers = [1, 2, 11, 12, 16, 34, 36, 58, 60, 61, 62, 67, 83]

    # sau đó tạo một danh sách có tên là files bằng cách nối đường dẫn của các tệp tfrecords trong 
    # thư mục data_path của từ điển params với các số trong danh mục numbers . Các tệp tfrecord này chứa các văn bản 
    # từ nguồn OpenWebtext 
    files = [os.path.join(params['data_path'], "openwebtext-newspaper_{}.tfrecord" .format(str(i)))  for i in numbers]


    # cuối cùng hàm này sẽ gọi một hàm khác có tên là bpe_text với các tham số là batch_size 
    # files, amount, iterations, stitch và batch . Hàm này se sẽ trả về một danh sách văn bản được
    # mã hóa bằng phương pháp byte pair encoding 
    return bpe_text(params["batch_size"], files, amount=params["n_ctx"], iterations=params["iterations"], stitch=2, batch=batch)



# Thiết lập phương thức kết hợp lấy văn bản 70 % trong đó thiên về việc lấy văn bản dài hơn 
# tham số params là một từ điển chứa các thông tin cấu hình cho hàm , như batch_size , interation và data_path
# tham số eval là một giá trị boolean xác định xem hàm này có được sử dụng cho mục đích 
# đánh giá hay không 
def openwebtext_logbiased(params , eval=False):
    # lấy tất cả các tập văn bản từ 2 phương thức openwebtext(phương thức chứa taonf bộ văn bản) và 
    # openwebtext_long (phương thức chỉ lấy văn bản dài hơn 512 tokens) gán cho biến dataset
    datasets = [openwebtext(params , eval=eval, batch=False), openwebtext_long(params, eval=eval, batch=False)]
    # khởi tạo 1 danh sách với 2 tham số 0.3 và 0.7 
    weights = [0.3,0.7]

    # trích xuất từ danh sách nguồn dataset với 30 % văn bản và 70% văn bản dài hơn 512 tokens bằng phương thức 
    # tf.data.experimental.sample_fromdatasets gán nó cho biến dataset 
    dataset = tf.data.experimental.sample_from_datasets(datasets, weights=weights)
    # sử dụng phương thức batch để chia danh sách văn bản thành các batch size = params[batch]
    # với số lượng văn bản trong đối tượng dataset không chia hết cho kích thước batch , thì batch 
    # cuối cùng sẽ bị bỏ qua , do tham số drop_remaindr = True 
    #  sử dụng phương thức .prefetch để tìm nạp trước (tải trước )một số lượng văn bản 
    # với kích thước = params['interations'] để tăng tốc độ huấn luyện cho mô hình 
    dataset = dataset.batch(params["batch_size"], drop_remainder=True).prefetch(params["iterations"])

    # Trả về đối tượng dataset cuối cùng, đã được chia thành các batch và tải trước.
    return dataset

# A generic function to take in any tfrecords files filled with the correct BPE text
# Thiết lập một hàm chung để nhận bất kỳ tệp tfrecord nào chứa văn bản BPE chính xác
# từ điển params chứa các tham số batch_size , interations (số nguyên xác định số lượng văn bản đuọc nạp trước)
#  stitch: Một số nguyên, xác định số lượng văn bản được ghép lại thành một văn bản dài hơn.

def generic_text(params):
    # dataset: Một danh sách các cặp ([files], weight), trong đó [files] là một danh sách
    # các tệp tfrecords chứa các văn bản đã được mã hóa bằng phương pháp BPE,
    # còn weight là một số thực xác định tỉ lệ lấy mẫu từ [files]
    datasets = [bpe_text(params["batch_size"], dataset[0], amount=params["n_ctx"], iterations=params["iterations"], stitch=params["stitch"], batch=False)
                for dataset in params["dataset"]]
    # lấy ra giá trị tham số dataset từ bộ từ điển params gán cho weights
    weights = [dataset[1] for dataset in params["dataset"]]

    # tương tự như phương thức trên trích xuất danh sách các tệp văn bản ngẫu nhiên theo tỷ lệ 
    # weights từ danh sách các đối tượng dataset ban đầu 
    dataset = tf.data.experimental.sample_from_datasets(datasets, weights=weights)
    # chia danh scahs dataset mới thành các batch và tải trước một lượng để tăng tốc độ huấn luyện 
    dataset = dataset.batch(params["batch_size"], drop_remainder=True).prefetch(params["iterations"] * 2)

    #  Trả về đối tượng dataset cuối cùng, đã được chia thành các batch và tải trước.
    return dataset
  

# thiết lập phương thức xử lý nén văn bản 
def bpe_text(batch_size, files ,iterations, stitch, amount=1024, batch=True):
    # tạo một đối tượng dataset là bằng căchs sử dụng phương tức tf.data.Dataset.from_tensor_slices với tham số files
    # Hàm này sẽ tạo một đối tuowngj Dataset chứa các phần tử là tên các tệp tfrecord trong
    # danh sách files 
    dataset = tf.data.Dataset.from_tensor_slices(files)
    # Biến đổi danh sách dataset bằng cách sử dụng methood apply với hàm tf.data.experiment.parallel_interleave
    # có tham số tf.data.TFRecordDataaset. tạo ra một đối tượng dataset mói bằng cách đọc song song các tệp tfrecord
    # với số lượng tệp tối đa = 4 và cho phéo sự không đồng nhất trong thứ tự các phần tử 
    dataset = dataset.apply(tf.data.experimental.parallel_interleave(tf.data.TFRecordDataset, cycle_length=4, sloppy=True))

    # định nghĩa 1 hàm có tên _parse_function với tham số example_proto 
    # có chức năng giải mã một phần tử của đối tượng dataset, là một bản ghi tfrecord , 
    # thành 1 cặp text , length với text là văn bản được mã hóa bằng BPE và độ dài length củab nó
    def _parse_function(example_proto):
        # tạo mo0otj từ điển features với key = "hash" ; "text"
        # từ điển này xác định cấu trúc của một bản ghi tfrecord gôm 2 trường với kiểu 
        # dữ liệu của nó 
        features = {
            "hash" : tf.VarLenFeature(tf.string),
            "text" : tf.VarLenFeature(tf.int64)
        }
        # sử dụng một hàm có tên là tf.parse_sigle_example với accs tham số là example_proto và feature
        # để giải mã một bản ghi tfrecord thành một từ điển có tên là parsed_features, chứa 2 cặp khóa và
        # giá trị : Một spareTensor là một đối tượng biểu diễn tensor thưa ó ba thuộc tính là indices, values và dense_shape.
        parsed_features = tf.parse_sigle_example(example_proto, features)
        
        # trả về văn bản đã được phân tích và độ dài của văn bản 
        return parsed_features["text"] , parsed_features["text"].dense_shape[0]
    
    # Biến đổi phương thức dataset thành 1 phương thức mới bằng cách áp dụng hàm _parse_function 
    # cho danh sách dataset chứa các tệp tfrecord . với số lượng hàm được gọi song song là 1#
    # điều này có nghĩa là mỗi phần tử dataset sẽ là 1 cặp text ; length chứ không phải 1 bản ghi 
    dataset = dataset.map(_parse_function, num_parallel_calls=1).shuffle(1000* stitch)


    # stitch là một số nguyên xác định số lượng văn bản cần ghép lại 
    #  # Since samples can be less than the correct length, and TPUs don't like variable lengths, this function stitches together enough samples
    # to have a text at least 1024 tokens long. For this to work the stitch parameter must be correctly tuned so that
    # stitch * min(characters_in_text) >= amount
    """
        Vì các mẫu có thể nhỏ hơn độ dài chính xác và TPU không thích độ dài thay đổi 
            nên chức năng này sẽ ghép đủ các mẫu lại với nhau để có văn bản dài ít nhất 1024 mã thông báo.
            Để làm được việc này, tham số mũi khâu phải được điều chỉnh chính xác sao cho

        stitch * min(characters_in_text) >= amount
        Hàm này có chức năng là ghép các văn bản trong x thành một văn bản duy nhất, 
            với độ dài của mỗi văn bản được lưu trong y
    """
    def _stitch_text(x, y):
        # chuyển 1 spareTensor thành 1 tensor dày bằng hàm tf.spare.to_dense 
        # hàm này sẽ tạo một tensor mới có cùng kích thước với x nhưng không có phần tử thưa 
        x = tf.sparse.to_dense(x)
        # đingh nghĩa 1 hàm get_x với tham số i  có chức annwg lấy ra một văn bản từ x với 
        # chỉ số i , và cắt bỏ các phần tử thưa bằng cách sử dụng hàm tf.gather 
        # Hàm này sẽ tạo ra một tensor mới, chứa các giá trị của x[i] tại các chỉ số trong tf.range(y[i]). 
        # Điều này có nghĩa là hàm này sẽ trả về một văn bản có độ dài bằng với giá trị của y[i].
        def _get_x(i):
            return tf.gather(x[i], tf.range(y[i]))

        # gán giá trị của out = _get_x(0) điều này có nghĩa là out sẽ là văn bản đầu tiên trong x
        out = _get_x(0)
        # duyệt qua 1 danh sách từ 1 đến stitch - 1
        for i in range(1, stitch):
            # sử dụng hàm tf.concat để nói các tensor trong danh scahs theo chiều dọc và gán lại cho out 
            # Hàm này tạo ra một tensor mới chứa các giá trị của out một số 50256 vè kết quả của get_x(i)
            # theo thứ tự đó , số 50256 là mã BPE của ký tự đặc biệt dùng để ngăn cách văn bản 
            out = tf.concat([out, [50256], _get_x(i)], axis=0) # text1<|endoftext|>text2
            
        # trả về danh sách out 
        return out

    # Hack-y way to stitch together multiple texts
    # chia dnah sách dataset ban đầu thành cách batch số lượng văn bản trong đối tượng dataset không chia hết cho kích thước batch , thì batch 
    # cuối cùng sẽ bị bỏ qua , do tham số drop_remaindr = True . sử dụng map để áp dụng phương thức _stitch_text cho toàn bộ các batch data
    # để ghép các văn bản trong 1 batch thành 1 văn bản duy nhất cách nhau = ký tự đặc biệt với các đoạn độ dài = 1024 và số lượng phương thức 
    # được gọi song song tự thiết lập 
    dataset = dataset.batch(stitch, drop_remainder=True).map(_stitch_text, num_parallel_calls=tf.data.experimental.AUTOTUNE)

    # Sample 1024(+1) tokens from the stitched together text
    # Định nghĩa một hàm con có tên là _sample_text, với một tham số là x. 
    # Hàm này có chức năng là lấy mẫu một đoạn văn bản có độ dài 1024 tokens từ một văn 
    # bản dài hơn, và trả về một cặp (vals1, vals2), trong đó vals1 là đoạn văn bản được lấy mẫu, 
    # còn vals2 là đoạn văn bản kế tiếp của vals1.
    def _sample_text(x):
        # lấy ra kích thước x là một số nguyên cho biết số 
        # luowngj phần tử của x 
        s = tf.size(x)
        # tạo một số ngẫu nhiên 0 -> s - (amount + 1) kiểu int32
        r = tf.random.uniform([], maxval=s-(amount+1), dtype=tf.dtypes.int32)
        # tạo 2 tensor r1  và r2 từ tham số r và r1 + 1 đến r+amoun và r+1 + amount 
        r1 = tf.range(r, r+amount)
        r2 = tf.range(r+1, (r+1)+amount)

        # định hình laị 2 tensor r1  , r2 với các giá trị không thay đổi nhưng kích thước = [amount]
        # điều này làm cho trình biên dịch hài lòng vì TPU muốn có kích thước input cố định 
        r1 = tf.reshape(r1,[amount])
        r2 = tf.reshape(r2, [amount])

        # lấy ra các giá trị x theo chỉ số của danh sách r1 và r2
        vals1 = tf.gather(x, r1)
        vals2 = tf.gather(x, r2)
        # Trả về một cặp (vals1, vals2), là hai đoạn văn bản có độ dài 1024 tokens, được lấy mẫu từ x.
        return vals1, vals2
    
    # kiểm tả xem giá trị batch có rồn tại 
    if batch:
        # nếu có ta áp dụng phuuwong thức  tf.data.experimental.map_and_batch
        # với ahmf chức năng _sample_text để chia văn bản trong 1 batch thành các đoạn
        # mỗi đoạn 1024 tokens , với số lượt kêu gọi phương thức song song = auto 
        dataset = dataset.apply(tf.data.experimental.map_and_batch(
            map_func=_sample_text, batch_size=batch_size,
            num_parallel_calls=tf.data.experimental.AUTOTUNE,
            # nếu số lượng văn bản không chia hết cho batch thì batch cuối cùng sẽ bị bỏ qua 
            drop_remainder=True))
        # sau đó lặp các đối tượng dataset liên tục và tìm nạp trước mộtsố lượng batch 
        # = interations 
        dataset = dataset.repeat().prefetch(iterations)

    # nếu batch không tồn tại 
    else:
        dataset = dataset.map(_sample_text, num_parallel_calls=tf.data.experimental.AUTOTUNE).repeat()

    return dataset

In [ ]:
# Create a batch of text
def gpt2_pred_input(params, text=None):
    # import encoder mã hóa từ GPT2
    from models.gpt2 import encoder
    # mã hóa dường đãn encoder_path Hàm này sẽ trả về một đối tượng của lớp encoder.Encoder, 
    # là một bộ mã hóa BPE được khởi tạo từ một tệp json chứa từ vựng và các quy tắc mã hóa.
    enc = encoder.get_encoder(params["encoder_path"])
    # sử dụng bộ mã hóa BPE để mã hóa văn bản 
    tokens = enc.encode(text)
    # kiểm tra xem số lượng token trong danh sách có > 1024 
    if len(tokens) > 1024:
        # nếu lớn hơn ta chia lại danh sách 
        tokens = tokens[:1024]
    # Tạo một tensor có tên là t, bằng cách sử dụng một hàm có tên là tf.broadcast_to,
    # với các tham số là tokens và [params["batch_size"], len(tokens)]
    #  Hàm này sẽ tạo ra một tensor mới, có cùng giá trị với tensor đầu tiên, nhưng có kích thước được xác định bởi tensor thứ hai.
    #  tensor t sẽ có kích thước là params["batch_size"] nhân với len(tokens), và mỗi hàng của tensor t sẽ chứa danh sách tokens.
    t = tf.broadcast_to(tokens, [params["batch_size"], len(tokens)])
    # tạo ra một đối tượng dataset chứa một phần tử duy nhất là tensor t.
    dataset = tf.data.Dataset.from_tensors(t)
    # Trả về đối tượng dataset, là một tập hợp các tokens đã được mã hóa bằng bộ mã hóa BPE của GPT-2.
    return dataset